In [ ]:
import xarray as xr
import os
import pathlib
import numpy as np
import xesmf as xe

In [ ]:
def sel_tropics(x):
    """subset for data in tropics"""

    ## find grid points in tropics
    in_tropics = (x.lat < 20) & (x.lat > -20)
    in_tropics.load()

    ## get index to select
    y_idx = in_tropics.any("x")

    return x.isel(y=y_idx)


def regrid(x):
    """regrid data to match other oras data"""

    ## load ref grid
    grid = xr.open_dataset(
        ORAS_FP / "tos_control_monthly_highres_2D_1958-2020_CONS_v0.1.nc"
    )
    grid = grid.isel(time_counter=0).drop_vars("time_counter").compute()

    ## regrid
    regridder = xe.Regridder(x, grid, "bilinear")
    return regridder(x)

#### load data

In [ ]:
## path to data
ORAS_FP = pathlib.Path(os.environ["DATA_FP"], "oras5")
nhf_fp = ORAS_FP / "nhf"

## open data
data_cons = xr.open_mfdataset(np.sort(list(nhf_fp.glob("*CONS*.nc"))))
data_oper = xr.open_mfdataset(np.sort(list(nhf_fp.glob("*OPER*.nc"))))

## update coords
data_cons["lat"] = data_cons["nav_lat"].isel(time_counter=0)
data_cons["lon"] = data_cons["nav_lon"].isel(time_counter=0)
data_cons = data_cons.drop_vars(["nav_lat", "nav_lon"])
data_oper = data_oper.rename({"nav_lat": "lat", "nav_lon": "lon"})

## trim
data_cons = sel_tropics(data_cons)
data_oper = sel_tropics(data_oper)

print("Loading")
data_cons.load()
data_oper.load();

#### regrid

In [ ]:
## regrid consolidated
data_cons_regrid = regrid(data_cons)
data_oper_regrid = regrid(data_oper)

#### Merge and save to file

In [ ]:
## concatenate and add attributes
data = xr.concat([data_cons_regrid, data_oper_regrid], dim="time_counter")
data.attrs = data_cons["sohefldo"].attrs

## save
data.to_netcdf(ORAS_FP / "nhf_1958-2020.nc")